In [1]:
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px

In [2]:
data = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv')
data

,Date,Year,Month,Recession,Consumer_Confidence,Seasonality_Weight,Price,Advertising_Expenditure,Competition,GDP,Growth_Rate,unemployment_rate,Automobile_Sales,Vehicle_Type,City
0,1/31/1980,1980,Jan,1,108.24,0.50,27483.571,1558,7,60.223,0.010000,5.4,456.0,Supperminicar,Georgia
1,2/29/1980,1980,Feb,1,98.75,0.75,24308.678,3048,4,45.986,-0.309594,4.8,555.9,Supperminicar,New York
2,3/31/1980,1980,Mar,1,107.48,0.20,28238.443,3137,3,35.141,-0.308614,3.4,620.0,Mediumfamilycar,New York
3,4/30/1980,1980,Apr,1,115.01,1.00,32615.149,1653,7,45.673,0.230596,4.2,702.8,Supperminicar,Illinois
4,5/31/1980,1980,May,1,98.72,0.20,23829.233,1319,4,52.997,0.138197,5.3,770.4,Smallfamiliycar,California
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523,8/31/2023,2023,Aug,0,103.36,0.25,27852.993,1793,6,57.169,0.764155,2.6,1579.6,Executivecar,New York
524,9/30/2023,2023,Sep,0,101.55,0.07,21183.704,1028,5,59.315,0.036180,2.5,1123.4,Smallfamiliycar,California
525,10/31/2023,2023,Oct,0,124.66,0.12,15975.589,1148,9,19.472,-2.046169,2.5,1685.9,Sports,California
526,11/30/2023,2023,Nov,0,97.09,0.25,16862.288,4850,5,27.904,0.302179,2.9,2124.6,Smallfamiliycar,Georgia


In [3]:

year_list= [i for i in range(1980, 2024, 1)]

In [4]:
app = dash.Dash(__name__)

In [14]:

recession_data = data[data['Recession'] == 1]
yearly_rec=recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
R_chart1=dcc.Graph(figure=px.line(yearly_rec, 
                x=yearly_rec['Year'],
                y=yearly_rec['Automobile_Sales'],
                title="AVG_Automobile_Sales_by_year"))

type_sold=recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
R_chart2=dcc.Graph(figure=px.bar(type_sold, 
                x=type_sold['Vehicle_Type'],
                y=type_sold['Automobile_Sales'],
                title="AVG_Automobile_Sales_by_Vehicle_Type"))

exp_rec= recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum()
sizes=exp_rec.values
labels=exp_rec.index
R_chart3=dcc.Graph(figure=px.pie(values=sizes ,
                names=labels,
                title="Total_Expenditure_share_by_Vehicle_type"))

unemployment_rate=recession_data.groupby(['unemployment_rate','Vehicle_Type'])['Automobile_Sales'].sum().reset_index()
R_chart4=dcc.Graph(figure=px.bar(unemployment_rate, 
                x=unemployment_rate['unemployment_rate'],
                y=unemployment_rate['Automobile_Sales'],
                color=unemployment_rate['Vehicle_Type'],
                title='The_Effect_of_Unemployment_Rate_on_Vehicle_Type_and_Sales'))


In [19]:

yas= data.groupby('Year')['Automobile_Sales'].mean().reset_index()
Y_chart1=dcc.Graph(figure=px.line(yas,
              x=yas['Year'],
              y=yas['Automobile_Sales'],
              title='Avg_Automobile_Sales_By_Year'))
monthly_sales=data.groupby('Month')['Automobile_Sales'].sum().reset_index()
Y_chart2=dcc.Graph(figure=px.line(monthly_sales,
              x=monthly_sales['Month'],
              y=monthly_sales['Automobile_Sales'],
              title='Total_Automobile_Sales_By_Month'))
avr_vdata=data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
Y_chart3=dcc.Graph(figure=px.bar(x=avr_vdata['Vehicle_Type'],
               y=avr_vdata['Automobile_Sales'],
               title='Average Vehicles Sold by Vehicle Type in the year '))
total_adv=data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum()
sizes=total_adv.values
labels=total_adv.index
Y_chart4=dcc.Graph(figure=px.pie(values=sizes,
                names=labels,
                title="Total_Expenditure_by_Vehicle_type"))

In [21]:

app.layout=html.Div([html.H1("Automobile Statistics Dashboard",
                             style={'textAlign': 'center', 'color': '#503D36', 'font-size': 24}),
                     html.Div([html.Label("Select Statistics:"),
                               dcc.Dropdown(id='dropdown-statistics', 
                                             options=[
                                                 {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
                                                 {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}],
                                             placeholder='Select a report type',
                                             value="Select Statistics",
                                             style={'width':'80%', 'padding':'3px', 'font-size':'20px', 'text-align-last':'center'}),
                              ]),
                     html.Div(dcc.Dropdown(id='select_year',
                                              options=[{'label': i, 'value': i} for i in year_list],
                                              placeholder='Select a Year',
                                            style={})),
                    html.Div(id='output_container', className="chart_grid", style={"display":"flex"}),
                    html.Div(children=[html.Div(children=R_chart1),html.Div(children=R_chart2)],style={'display': 'flex'}),
                    html.Div(children=[html.Div(children=R_chart3),html.Div(children=R_chart4)],style={'display': 'flex'})


                    
                    
                    ])
                               
                     
@app.callback(Output(component_id='select_year', component_property='disabled'),
              Input(component_id='dropdown-statistic', component_property='value'))
def update_input_container(input_value):
    if input_value =='Yearly Statistics': 
        return False
    else: 
        return True
    
@app.callback(Output(component_id='output_container', component_property='children'),
             [Input(component_id='dropdown-statistic', component_property='value'),
              Input(component_id='select_year', component_property='value')])   
def output_container(drop, year):
    if drop == 'Recession Period Statistics':
        recession_data = data[data['Recession'] == 1]
        yearly_rec=recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        R_chart1=dcc.Graph(figure=px.line(yearly_rec, 
                        x=yearly_rec['Year'],
                        y=yearly_rec['Automobile_Sales'],
                        title="AVG_Automobile_Sales_by_year"))

        type_sold=recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        R_chart2=dcc.Graph(figure=px.bar(type_sold, 
                        x=type_sold['Vehicle_Type'],
                        y=type_sold['Automobile_Sales'],
                        title="AVG_Automobile_Sales_by_Vehicle_Type"))

        exp_rec= recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum()
        sizes=exp_rec.values
        labels=exp_rec.index
        R_chart3=dcc.Graph(figure=px.pie(values=sizes ,
                        names=labels,
                        title="Total_Expenditure_share_by_Vehicle_type"))

        unemployment_rate=recession_data.groupby(['unemployment_rate','Vehicle_Type'])['Automobile_Sales'].sum().reset_index()
        R_chart4=dcc.Graph(figure=px.bar(unemployment_rate, 
                        x=unemployment_rate['unemployment_rate'],
                        y=unemployment_rate['Automobile_Sales'],
                        color=unemployment_rate['Vehicle_Type'],
                        title='The_Effect_of_Unemployment_Rate_on_Vehicle_Type_and_Sales'))
        
        return [ html.Div(className='chart-item', children=[html.Div(children=R_chart1),html.Div(children=R_chart2)],style={'display': 'flex'}),
                 html.Div(className='chart-item', children=[html.Div(children=R_chart3),html.Div(children=R_chart4)],style={'display': '....'})
                 ]
    elif drop =='Yearly Statistics':
            yearly_data = data[data['Year'] ==int(year)]
            yas= yearly_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
            Y_chart1=dcc.Graph(figure=px.line(yas,
                          x=yas['Year'],
                          y=yas['Automobile_Sales'],
                          title='Avg_Automobile_Sales_By_Year'))
            monthly_sales=yearly_data.groupby('Month')['Automobile_Sales'].sum().reset_index()
            Y_chart2=dcc.Graph(figure=px.line(monthly_sales,
                          x=monthly_sales['Month'],
                          y=monthly_sales['Automobile_Sales'],
                          title='Total_Automobile_Sales_By_Month'))
            avr_vdata=yearly_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
            Y_chart3=dcc.Graph(figure=px.bar(x=avr_vdata['Vehicle_Type'],
                           y=avr_vdata['Automobile_Sales'],
                           title='Average Vehicles Sold by Vehicle Type in the year {}'.format(input_year)))
            total_adv=yearly_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum()
            sizes=total_adv.values
            labels=total_adv.index
            Y_chart4=dcc.Graph(figure=px.pie(values=sizes,
                            names=labels,
                            title="Total_Expenditure_by_Vehicle_type"))


            return [ html.Div(className='chart-item', children=[html.Div(children=Y_chart1),html.Div(children=Y_chart2)],style={'display': 'flex'}),
                     html.Div(className='chart-item', children=[html.Div(children=Y_chart3),html.Div(children=Y_chart4)],style={'display': '....'})
                 ]
    else:
        return None



if __name__=='__main__':
             app.run_server(jupyter_mode='tab')
                     

Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>